<a href="https://colab.research.google.com/github/AleemKhan12/AleemKhan12/blob/V0.14/Attendance_Tracking_Calendar_Export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Attendance Tracking by using Calendar and Audit Log Data

#Revision notes V0.14: Chaged google authentication service to use Google Auth

'''
ISSUES


'''


"\nISSUES\n1. Reading google sheet for Audit data returning wrong values\n2. Test time conversion accuracy. Punctuality returning wrong values\n4. Some calendar event id's not attenuating resulting in missed attendance\n\n"

# CONNECTING TO GOOGLE DRIVE

In [ ]:
#connect colab to drive

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)


In [ ]:
#Imports
import pandas as pd
from datetime import date, timedelta
import pytz
from numpy.core.numeric import NaN
import datetime
import math



#IMPORTANT GLOBAL VARIABLES

In [ ]:
mentor_email = 'mentor@villagebookbuilders.org' #Mentor program email address
oracle_data       = 0                           #Data read from Oracle Database SQL Query
google_data       = 0                           #Data read from Google Audit Log
active_sessions   = 0                           #Active sessions recorded in Oracle db based on end date
total_mentors     = 0                           #List of all mentors names in oracle db
oracle_mentor_emails = []                       #list of all mentor emails in oracle db
active_mentors    = 0                           #Mentor names that joined at least one session 
active_mentor_emails = []                       #List of mentor emails that joined at least one session
mentor_programs   = []                          #Names of all mentor programs stored in oracle database
mentors_not_registered = []                     #Names of all active mentors not logged in oracle database
time_zone = 'America/Port_of_Spain'                         #google_data['Date'][1][len(google_data['Date'][1])-3:] #TODO: change this hard coded time zone
start = 0                                       #start date
end = 0                                         #end date
mentor_identifier = 'Mentor'                    #Mentor identifier
mentee_identifier = 'Student'                   #Mentee identifier
cal_id_len = 26                                 #usable calendar event id string length
google_resource_identifier = '@resource.calendar.google.com'
fire_flies_email = 'fred@fireflies.ai'
attendance_spreadsheet_key = '1zdKt4k-d9tvi5pm_nyQj5KfdLhVcJSDL5ekrXwfkR1M'
productivity_spreadsheet_key = '1ag6astOJXNwCJRz3vVu3gkGa_aMEi4E7VaeCj5ObbSk'
attendance_info_url = f'https://docs.google.com/spreadsheets/d/{attendance_spreadsheet_key}/edit#gid=0'
productivity_info_url = f'https://docs.google.com/spreadsheets/d/{productivity_spreadsheet_key}/edit#gid=0'
mentee_comp_url = 'https://docs.google.com/spreadsheets/d/1JcJ99pX-pjMTCkFrLwjpaCJ_1FcJ2xZ7UqgBJHGFl9I/edit#gid=925063253'
corporate_partner_url = 'https://docs.google.com/spreadsheets/d/1K9FtfVv-lzU0eQE4V27Qdb3zuj2t9d-WOxs2ZAA04kw/edit#gid=0'
program_director_name = ['adeiso ghana', 'ahero kenya', 'biwi, malawi vbb mentor program', 'charity centre, zambia vbb mentor program', 'chebosi kenya', 'kadzakalowa malawi', 'maragoli kenya', 'mukono uganda', 'vima ghana'] #Actor names of program director accounts

# READING DATA SOURCES

In [ ]:
#IMPORTANT: THE ONLY THINGS THAT YOU NEED TO CHANGE ARE THE GOOGLE SHEET URL's FOR THE CALENDAR DATA AND GOOGLE MEET AUDIT LOG DATA
calendar_sheet_url = 'https://docs.google.com/spreadsheets/d/1NHgcj_W104Ic3zHNXU_mq4D18TaGep7LgUukZtne1BQ/edit#gid=0'
audit_log_sheet_url = 'https://docs.google.com/spreadsheets/d/14fKIVCvaQ8PTCHFStQYHWXJs663-Jqf83ED2KpUmz6s/edit#gid=0'

In [ ]:
#Read Data Sources as pandas data frames

#previous attendance data
sheet_url = attendance_info_url
attendance_path = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
#calendar_path = '/content/gdrive/MyDrive/Projects/Mentor_Dashboard_V2/3. Execution/SRC Code/Data/Calendar Data Export/Calendar Export UTC - Sheet1.csv' 
attendance_data = pd.read_csv(attendance_path, on_bad_lines='skip')

#previous productivity data
sheet_url = productivity_info_url
productivity_path = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
#calendar_path = '/content/gdrive/MyDrive/Projects/Mentor_Dashboard_V2/3. Execution/SRC Code/Data/Calendar Data Export/Calendar Export UTC - Sheet1.csv' 
productivity_data = pd.read_csv(productivity_path, on_bad_lines='skip')

#Calendar Data
sheet_url = calendar_sheet_url
calendar_path = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
#calendar_path = '/content/gdrive/MyDrive/Projects/Mentor_Dashboard_V2/3. Execution/SRC Code/Data/Calendar Data Export/Calendar Export UTC - Sheet1.csv' 
calendar_data = pd.read_csv(calendar_path, on_bad_lines='skip')

#Google Audit Log Data
sheet_url = audit_log_sheet_url
google_path = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
#google_path = '/content/gdrive/MyDrive/Projects/Mentor_Dashboard_V2/3. Execution/SRC Code/Data/Google Audit Log/3 6 2022 - 3 12 2022/3_6_2022 - 3_12_2022.csv'
google_data = pd.read_csv(google_path, on_bad_lines='skip')


#Mentee Computer Emails
sheet_url = mentee_comp_url
mentee_email_path = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
#mentee_email_path = '/content/gdrive/MyDrive/Projects/Mentor_Dashboard_V2/3. Execution/SRC Code/Data/mentee email/Copy of Village Library Information - computer email.csv'
mentee_computer_emails = pd.read_csv(mentee_email_path, on_bad_lines='skip')

#Corporate Parners
sheet_url = corporate_partner_url
corporate_partners_path = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
#corporate_partners_path = '/content/gdrive/MyDrive/Projects/Mentor_Dashboard_V2/3. Execution/SRC Code/Data/Corporate Organizations/Corporate Partners.csv'
corporate_partners = pd.read_csv(corporate_partners_path, on_bad_lines='skip')



In [ ]:
print(attendance_data.shape, productivity_data.shape, calendar_data.shape, google_data.shape, mentee_computer_emails.shape, corporate_partners.shape)

(15305, 20) (3785, 9) (1507, 11) (524, 31) (146, 8) (3, 2)



# CLEANING DATA

In [ ]:
#Calendar Data
calendar_data = calendar_data.dropna() #remove all empty sessions
calendar_data = calendar_data.drop_duplicates()
calendar_data = calendar_data[(~calendar_data['Participant Identifier'].str.contains(google_resource_identifier)) & (calendar_data['Participant Identifier'] != fire_flies_email)]

#Google Data
google_data = google_data.dropna(subset=['Date', 'Meeting code', 'Actor', 'Organizer email', 'Duration (seconds)', 'Actor name', 'Calendar event ID']) #clean google data
google_data = google_data.drop_duplicates()
google_data = google_data[google_data['Duration (seconds)'] > 0] #drop all calls with 0 duration
google_data = google_data[google_data['Organizer email'] == mentor_email] #only consider mentor sessions
google_data = google_data[google_data['Actor name'] != 'Mentor Program'] #drop all entries for mentor program

#Mentee Email data
mentee_computer_emails = mentee_computer_emails.drop_duplicates(['COMPUTER_EMAIL'])

#Corporate Parner data
corporate_partners = corporate_partners.dropna()

In [ ]:
print(calendar_data.shape, google_data.shape)

(1082, 11) (467, 31)


In [ ]:
#DATE CONVERSIONS
#Only run this once. Running more times will cause an error. To fix re-read data sources
def conv_to_datetime(a):
  utc_timezone = pytz.timezone('UTC')
  if isinstance(a, datetime.date):
    return a
  try:
    #Convert calendar export dates
    date = datetime.datetime.strptime(a, '%m/%d/%Y %H:%M:%S')
    date = date.replace(tzinfo = pytz.timezone('UTC')) 
    return date
  except:
    try:
      #Convert Audit log dates 
      date = datetime.datetime.strptime(a, '%Y-%m-%dT%H:%M:%SZ')
      date = date.replace(tzinfo = pytz.timezone('UTC')) 
      return  date
    except:
      date = datetime.datetime.strptime('', '')
      date = date.replace(tzinfo = utc_timezone)
      return date



In [ ]:
#convert attendance data times to date time
attendance_data['Start Datetime'] = attendance_data['Event Start UTC'].apply(conv_to_datetime)
attendance_data['End Datetime'] = attendance_data['Event End UTC'].apply(conv_to_datetime)

In [ ]:
#converting calendar dates to datetime objs
calendar_data['Start Datetime'] = calendar_data['Event Start UTC'].apply(conv_to_datetime)
calendar_data['End Datetime'] = calendar_data['Event End UTC'].apply(conv_to_datetime)



In [ ]:
#Getting usable substring from Calendar Event ID
calendar_data['Calendar event ID'] = calendar_data['Calendar event ID'].apply(lambda x: x[:cal_id_len])

In [ ]:
#Convert google dates to datetime UTC
#google_data['Date'] = google_data['Date'].apply(lambda x: x[:-4]) #remove time zone
google_data['Date'] = google_data['Date'].apply(conv_to_datetime)

In [ ]:
#Create new column with just dates
google_data['String_Date'] = google_data['Date'].apply(lambda x: x.strftime("%m/%d/%Y"))
calendar_data['String_Date'] = calendar_data['Start Datetime'].apply(lambda x: x.strftime("%m/%d/%Y"))

In [ ]:
#lower participant emails for accurate searches
google_data['Actor'] = google_data['Actor'].apply(lambda x: x.lower())
calendar_data['Participant Identifier'] = calendar_data['Participant Identifier'].apply(lambda x: x.lower())

In [ ]:
#Drop duplicate sessions caused by people signing in from various devices at the same time
#google_data = google_data.drop_duplicates(subset = ['Participant Identifier', 'Calendar event ID', 'String_Date', 'IP address'])

In [ ]:
#list of mentee computer emails
mentee_computer_emails_list = mentee_computer_emails['COMPUTER_EMAIL'].tolist()


#TESTS

In [ ]:
#Testing the total number of unique mentors for this weekly dataset
participants = google_data.drop_duplicates(subset = ['Actor'])
participants = participants[participants['String_Date'] != '03/13/2022']

In [ ]:
names = participants['Actor']
student_emails = mentee_computer_emails['COMPUTER_EMAIL']
len(names[~names.isin(student_emails)].tolist())

176

In [ ]:
#Today's date
today_date = datetime.datetime.today().replace(tzinfo = pytz.timezone(time_zone))

#Start and end dates for schema 1
dates_list = sorted(google_data['Date'].tolist())
start1, end1 = dates_list[0], dates_list[-1] 

#Start and end dates for schema 2
end, start = datetime.datetime(2022, 2, 27).astimezone(pytz.timezone("utc")), datetime.datetime(2022, 3, 5).astimezone(pytz.timezone("utc"))



In [ ]:
print(start1, end1)
print(start, end)

2022-03-27 11:23:48+00:00 2022-04-02 19:48:36+00:00
2022-03-05 00:00:00+00:00 2022-02-27 00:00:00+00:00


#ATTENDANCE AND DURATION TRACKING 

In [ ]:
calendar_data['Attendance'] = [0]*len(calendar_data)
calendar_data['Duration'] = [0]*len(calendar_data)
calendar_data['User Type'] = [0]*len(calendar_data)
calendar_data['Organization'] = [0]*len(calendar_data)
#calendar_data['Punctuality'] = [NaN]*len(calendar_data)

In [ ]:
from os import putenv
for index, meeting in calendar_data.iterrows():
  attendance = 0
  duration = 0
  meeting_data = google_data[(google_data['Actor'] == meeting['Participant Identifier']) & (google_data['Calendar event ID'].str.contains(meeting['Calendar event ID'])) & (google_data['String_Date'] == meeting['String_Date'])]

  if (len(meeting_data) > 0):
    attendance = 1
    duration = sum(meeting_data['Duration (seconds)'].tolist())/60 #duration in minutes

  calendar_data['Attendance'][index] = attendance
  calendar_data['Duration'][index] = duration
  calendar_data['User Type'][index] = mentee_identifier if meeting['Participant Identifier'] in mentee_computer_emails_list else mentor_identifier
 

  #Assigning corporate partners
  partner_name = 'Public/Chapter'
  for i, partner in corporate_partners.iterrows():
    if partner['Email Identifier'] in meeting['Participant Identifier']:
      partner_name = partner['Organization']
      break;

  calendar_data['Organization'][index] = partner_name



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/l

In [ ]:
calendar_data['Missed_2'] = [0]*len(calendar_data)
calendar_data['Missed_3'] = [0]*len(calendar_data)

In [ ]:
#TODO: Write to complete list of attendance data here

#FINDING MENTOR THAT MISSED LAST 2 OR 3 SESSIONS

In [ ]:
#Identify mentors that have missed 2 or 3 of their last sessions. 

In [ ]:
for index, meeting in calendar_data.iterrows():

  #Get previous meetings regarless of id
  #TODO: Read from complete list of attendance data here, also find how to return max 3 entires 
  previous_meetings_1 = attendance_data[(attendance_data['Start Datetime'] <= meeting['Start Datetime']) & (attendance_data['Participant Identifier'] == meeting['Participant Identifier'])]
  previous_meetings_2 = calendar_data[(calendar_data['Start Datetime'] <= meeting['Start Datetime']) & (calendar_data['Participant Identifier'] == meeting['Participant Identifier'])] # & (calendar_data['Calendar event ID'] == meeting['Calendar event ID'])]
  
  previous_meetings = pd.concat([previous_meetings_1, previous_meetings_2])

  if (len(previous_meetings)>=3):
    last_3 = 0 if sum(previous_meetings['Attendance'].tolist()[-3:]) else 1
    last_2 = 0 if sum(previous_meetings['Attendance'].tolist()[-2:]) else 1
  elif (len(previous_meetings) == 2):
    last_3 = 0
    last_2 = 0 if sum(previous_meetings['Attendance'].tolist()) else 1
  else:
    last_3 = 0
    last_2 = 0
  
  #updating data frame 
  calendar_data['Missed_2'][index] = last_2
  calendar_data['Missed_3'][index] = last_3

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
print(attendance_data.shape, calendar_data.shape)

(15305, 20) (1082, 20)


In [ ]:
#Concat new and old data
attendance_data_new = pd.concat([attendance_data, calendar_data])
attendance_data_new = attendance_data_new.drop_duplicates(subset=['String_Date', 'Calendar event ID', 'Participant Identifier'], keep='last')

In [ ]:
print(attendance_data_new.shape)

(16387, 20)


In [ ]:
#TODO: merge actor names to calendar_data based on participant identifier
# actor_names = google_data[['Participant Identifier', 'Actor name']].drop_duplicates(subset = ['Participant Identifier'])
# print(actor_names.shape)
# actor_names = calendar_data[['Participant Identifier']].drop_duplicates(subset = ['Participant Identifier'])
# print(actor_names.shape)

# #Merge actor names
# result = calendar_data#pd.merge(calendar_data, actor_names, on='Participant Identifier')

# STORING ATTENDANCE DATA

In [ ]:
#Create data frame and store
#TODO: write to complete list of all attendance data; over write any redundant data
file_name = 'all_attendance_data.csv'
attendance_data_new.to_csv(file_name, index = False)

#FINDING PRODUCTIVE AND ALONE MEETINGS

In [ ]:
#XOR function
def xor(a, b):
  if((a and b) or (not a and not b)):
    return False
  else:
    return True

In [ ]:
#Convert list to string 
def listToString(s): 
    
    # initialize an empty string
    str1 = "" 
    
    # traverse in the string  
    for ele in s: 
        str1 += ele + ", "  
    
    # return string  
    return str1

In [ ]:
#A productive meeting will be a meeting in which at least one mentor and one mentee attended

In [ ]:
#Get all meetings per day 
productive_meetings  = calendar_data[['String_Date', 'Start Datetime' ,'Calendar event ID', 'Program Name']].drop_duplicates(subset=['String_Date', 'Calendar event ID'])
productive_meetings['Productive'] = [0]*len(productive_meetings)
productive_meetings['Alone'] = [0]*len(productive_meetings)
productive_meetings['Absent Type'] = [NaN]*len(productive_meetings)
productive_meetings['Absent Identifier'] = [NaN]*len(productive_meetings)

In [ ]:
#Get meeting start hour from datetime
productive_meetings['Meet Time'] = productive_meetings['Start Datetime'].apply(lambda x: x.strftime('%H:%M'))

In [ ]:
#For each meeting in each day see if the meeting was productive

for index, meeting in productive_meetings.iterrows():
  
  mentor_attendance = sum(calendar_data[(calendar_data['String_Date'] == meeting['String_Date']) & (calendar_data['Calendar event ID'].str.contains(meeting['Calendar event ID'])) & (calendar_data['User Type'] == mentor_identifier)]['Attendance'].tolist())
  mentee_attendance = sum(calendar_data[(calendar_data['String_Date'] == meeting['String_Date']) & (calendar_data['Calendar event ID'].str.contains(meeting['Calendar event ID'])) & (calendar_data['User Type'] == mentee_identifier)]['Attendance'].tolist())

  #if session was productive
  if(mentor_attendance and mentee_attendance):
    productive_meetings['Productive'][index] = 1
    productive_meetings['Absent Type'][index] = NaN

  #If only one particpant showed
  elif(xor(mentor_attendance, mentee_attendance)):
    #if only one participant joined, record as Alone Session
    productive_meetings['Alone'][index] = 1      

    #get absentee type
    if mentee_attendance:
      productive_meetings['Absent Type'][index] = mentor_identifier
      productive_meetings['Absent Identifier'][index] = listToString(calendar_data[(calendar_data['String_Date'] == meeting['String_Date']) & (calendar_data['Calendar event ID'].str.contains(meeting['Calendar event ID'])) & (calendar_data['User Type'] == mentor_identifier)]['Participant Identifier'].tolist())
    else:
      if mentor_attendance:
        productive_meetings['Absent Type'][index] = mentee_identifier
        productive_meetings['Absent Identifier'][index] = listToString(calendar_data[(calendar_data['String_Date'] == meeting['String_Date']) & (calendar_data['Calendar event ID'].str.contains(meeting['Calendar event ID'])) & (calendar_data['User Type'] == mentee_identifier)]['Participant Identifier'].tolist())

  #if no one showed
  if(not productive_meetings['Productive'][index] and not productive_meetings['Alone'][index]):
    productive_meetings['Absent Type'][index] = NaN

      
      #productive_meetings['Absent Type'][index] = (mentor_identifier if mentee_attendance else (mentee_identifier if mentor_attendance else NaN))
    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopy

In [ ]:
#Concat old and new data
productivity_data_new = pd.concat([productivity_data, productive_meetings])
productivity_data_new = productivity_data_new.drop_duplicates(['String_Date', 'Calendar event ID'], keep='last')

In [ ]:
print(productivity_data_new.shape, productivity_data.shape, productive_meetings.shape)

(4052, 9) (3785, 9) (267, 9)


# STORING PRODUCTIVE MEETINGS

In [ ]:
#Create data frame and store
#TODO: write to complete list of all attendance data; over write any redundant data
file_name = 'all_productivity_data.csv'
productivity_data_new.to_csv(file_name, index = False)

#SAVING DATAFRAMES AS GOOGLE SHEETS

In [ ]:
#Sheet Info
spreadsheet_key = attendance_spreadsheet_key
workbook = gc.open_by_key(spreadsheet_key)

#Converting datetimes to string
attendance_data_new['Start Datetime'] = attendance_data_new['Start Datetime'].astype(str)
attendance_data_new['End Datetime'] = attendance_data_new['End Datetime'].astype(str)
calendar_data.fillna('', inplace=True)


#Writing dataframe to google sheet
workbook.values_update(
  'sheet1!A1',
  params={
      'valueInputOption': 'USER_ENTERED'
  },
  body={
      'values': [attendance_data_new.columns.values.tolist()] + attendance_data_new.values.tolist()
  }
)

{'spreadsheetId': '1zdKt4k-d9tvi5pm_nyQj5KfdLhVcJSDL5ekrXwfkR1M',
 'updatedCells': 327760,
 'updatedColumns': 20,
 'updatedRange': 'Sheet1!A1:T16388',
 'updatedRows': 16388}

In [ ]:
#Sheet Info
spreadsheet_key = productivity_spreadsheet_key
workbook = gc.open_by_key(spreadsheet_key)

#Replace all NaN's with empty string
productivity_data_new.fillna('', inplace=True)
productivity_data_new['Start Datetime'] = productivity_data_new['Start Datetime'].astype(str)

#Writing to google sheet
workbook.values_update(
  'sheet1!A1',
  params={
      'valueInputOption': 'USER_ENTERED'
  },
  body={
      'values': [productivity_data_new.columns.values.tolist()] + productivity_data_new.values.tolist()
  }
)

{'spreadsheetId': '1ag6astOJXNwCJRz3vVu3gkGa_aMEi4E7VaeCj5ObbSk',
 'updatedCells': 36477,
 'updatedColumns': 9,
 'updatedRange': 'Sheet1!A1:I4053',
 'updatedRows': 4053}